In [1]:
import pandas as pd
import numpy as np

In [2]:
df_test = pd.read_csv('../dataset/processed/test/metrics/metrics_clean.csv')
df_test_label = pd.read_csv('../dataset/processed/test/label.csv')

df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])
df_test_label['timestamp'] = pd.to_datetime(df_test_label['timestamp'])

In [3]:
device = ['cpu', 'mem']
node = ['carts', 'catalogue', 'front_end', 'ip6', 
        'ip7', 'orders','payment', 'user', 'shipping']

In [4]:
df_test_label = df_test_label[df_test_label['label']==1]
df_test_label[['timestamp', 'root_cause']].to_csv('../dataset/processed/root_cause/injection_info.csv')
df_test_label.head()

,timestamp,label,root_cause
105,2022-03-26 10:15:00,1,cpu_usage_carts
106,2022-03-26 10:16:00,1,cpu_usage_carts
107,2022-03-26 10:17:00,1,cpu_usage_carts
108,2022-03-26 10:18:00,1,cpu_usage_carts
109,2022-03-26 10:19:00,1,cpu_usage_carts


In [5]:
for i in range(len(df_test_label)):
    timestamp = df_test_label['timestamp'].values[i]
    root_cause = df_test_label['root_cause'].values[i]
    
    info = pd.DataFrame(columns=['device', 'node', 'predict', 'real'])
    for d in device:
        for n in node:
            predict = np.mean(df_test[(df_test['timestamp']>=timestamp-pd.to_timedelta('15min')) & (df_test['timestamp']<=timestamp-pd.to_timedelta('10min'))][d+'_usage_'+n].values)
            real = df_test[df_test['timestamp']==timestamp][d+'_usage_'+n].values[0]
            info.loc[len(info)] = [d, n, predict, real]
    timestamp = str(timestamp).split('.')[0].replace('T', ' ')
    info.to_csv('../dataset/processed/root_cause/'+timestamp+'.csv', index=False)